# On codera la lecture d'ARN

In [3]:
DEBUT_LECTURE = "AUG"
STOPS = ["UAA", "UAG", "UGA"]

acide_amine = {
    "UUU" : "Phe",
    "UUC" : "Phe",

    "UUA" : "Leu",
    "UUG" : "Leu",
    "CUU" : "Leu",
    "CUC" : "Leu",
    "CUA" : "Leu",
    "CUG" : "Leu",

    "AUU" : "Ile",
    "AUC" : "Ile",
    "AUA" : "Ile",

    "AUG" : "Met",

    "GUU" : "Val",
    "GUC" : "Val",
    "GUA" : "Val",
    "GUG" : "Val",

    "UCU" : "Ser",
    "UCC" : "Ser",
    "UCA" : "Ser",
    "UCG" : "Ser",

    "CCU" : "Pro",
    "CCC" : "Pro",
    "CCG" : "Pro",
    "CCA" : "Pro",

    "ACU" : "Thr",
    "ACA" : "Thr",
    "ACG" : "Thr",
    "ACC" : "Thr",

    "GCU" : "Ala",
    "GCA" : "Ala",
    "GCC" : "Ala",
    "GCG" : "Ala",

    "UAU" : "Tyr",
    "UAC" : "Tyr",

    "CAU" : "His",
    "CAC" : "His",

    "CAA" : "Gln",
    "CAG" : "Gln",

    "AAU" : "Asn",
    "AAC" : "Asn",

    "AAA" : "Lys",
    "AAG" : "Lys",

    "GAU" : "Asp",
    "GAC" : "Asp",

    "GAA" : "Glu",
    "GAG" : "Glu",

    "UGU" : "Cys",
    "UGC" : "Cys",

    "UGG" : "Trp",

    "CGU" : "Arg",
    "CGA" : "Arg",
    "CGC" : "Arg",
    "CGG" : "Arg",
    "AGA" : "Arg",
    "AGG" : "Arg",

    "AGU" : "Ser",
    "AGC" : "Ser",

    "GGU" : "Gly",
    "GGC" : "Gly",
    "GGA" : "Gly",
    "GGG" : "Gly"

}


In [ ]:
# Trouver le début de la séquence
def find_start(arn):
    if arn[0:3] == DEBUT_LECTURE:
        return arn[3:]
    return find_start(arn[1:])

# Supprimer les lettres à la fin de l'ARN qui ne servent pas au calcul
def delete_end(arn):
    if arn[0:3] in STOPS:
        return ""
    return arn[0:3] + delete_end(arn[3:]) 

# ça retourne juste la séquence codante
def coding_sequence(arn):
    arn = find_start(arn)
    return delete_end(arn)

    
# Lire la séquence après avoir enlevé les éléments qu'on lit pas
def read_sequence(arn):
    if len(arn) <= 2 or arn[0:3] in STOPS:
        return ""
    return (acide_amine[arn[0:3]] + "-" + read_sequence(arn[3:]))

# def compare_two_sequences(arn1, arn2):
#     element1 = read_sequence()

In [29]:
# On lit notre ARN
def read_arn(arn):
    return read_sequence(find_start(arn))

In [33]:
ARN = "AUGUUAGAAAUUUCAUGGUGCCUUA"

print(read_arn(ARN))

Leu-Glu-Ile-Ser-Trp-Cys-Leu-


In [82]:
import random
def generate_letter():
    return random.choice(["A", "U", "G", "C"])

def generate_sequence():
    length = random.choice(range(50, 80))
    s = ""
    for i in range(0, length):
        s = s+generate_letter()
    return s

In [83]:
def add_letter(arn, index):
    letter = generate_letter()
    return arn[:index] + letter + arn[index:]

In [84]:
def delete_letter(arn, index):
    return arn[:index] + arn[index+1:]

In [85]:
def change_letter(arn, index):
    letter = generate_letter()
    return arn[:index] + letter + arn[index+1:]

In [ ]:
PROB_M=2
PROB_D=2
PROB_A=2

def changement(arn, index):
    current_length = len(arn)

    p = random.choice(range(0,100))
    if p < PROB_M:
        change_letter(arn, index)

    p = random.choice(range(0,100))
    if p < PROB_A:
        add_letter(arn, index)
        index = index + 1


    p = random.choice(range(0,100))
    if p < PROB_D:
        delete_letter(arn, index)

AUGUUAGAAAUUUCAUGGUGCCUUA


'GUGUUAGAAAUUUCAUGGUGCCUUA'

In [91]:
def sequence_changement(arn):
    if arn == '':
        return ''
    return changement(arn[0:1], 0) + sequence_changement(arn[1:])

print(ARN)
print(sequence_changement(ARN))

loop = 1
c = sequence_changement(ARN)

while read_arn(c) != read_arn(ARN):
    c = sequence_changement(ARN)  
    loop+1

print(loop)


AUGUUAGAAAUUUCAUGGUGCCUUA
AAAGUACAAAAAAACUGGCGCGGGCAC


RecursionError: maximum recursion depth exceeded